# Proximity Mapping Visualization
## Trentino Weather Stations & EEA Air Quality Stations

This notebook visualizes the proximity relationships between Trentino weather monitoring stations and nearby EEA air quality stations using interactive maps and statistical analysis.

## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

## 2. Load Proximity Mapping Data

In [ ]:
# Load the proximity mapping data
proximity_mapping = pd.read_csv('../output/trentino_eea_proximity_mapping.csv')

print("Proximity Mapping Data Shape:", proximity_mapping.shape)
print("\nFirst 10 rows:")
print(proximity_mapping.head(10))
print("\nData Summary:")
print(f"Unique Trentino Stations: {proximity_mapping['Trentino_Stazione'].nunique()}")
print(f"Unique EEA Stations: {proximity_mapping['EEA_Stazione'].nunique()}")
print(f"Total Station Pairs: {len(proximity_mapping)}")

## 3. Load Station Coordinates

In [ ]:
# Load Trentino stations with coordinates
trentino_data = pd.read_csv('../output/historical_weather_airPM_trentino.csv')
trentino_stations = trentino_data[['Stazione', 'Latitudine', 'Longitudine']].drop_duplicates()

# Load filtered EEA stations with coordinates
eea_data = pd.read_csv('../output/eea_filtered_by_proximity.csv')
eea_stations = eea_data[['Stazione', 'Latitudine', 'Longitudine']].drop_duplicates()

print("Trentino Stations:")
print(trentino_stations)
print(f"\nTotal: {len(trentino_stations)} unique stations")
print(f"\nEEA Stations (filtered):")
print(f"Total: {len(eea_stations)} unique stations")
print(f"\nEEA Data Shape: {eea_data.shape}")
print(f"Date Range: {eea_data['Data'].min()} to {eea_data['Data'].max()}")

## 4. Create Interactive Map Visualization

In [ ]:
# Create interactive map
center_lat = trentino_stations['Latitudine'].mean()
center_lon = trentino_stations['Longitudine'].mean()

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=9,
    tiles='OpenStreetMap'
)

# Add Trentino stations (blue markers)
for idx, row in trentino_stations.iterrows():
    folium.CircleMarker(
        location=[row['Latitudine'], row['Longitudine']],
        radius=10,
        popup=f"<b>Trentino:</b> {row['Stazione']}",
        color='blue',
        fill=True,
        fillColor='blue',
        fillOpacity=0.8,
        weight=2
    ).add_to(m)

# Add EEA stations (red markers)
for idx, row in eea_stations.iterrows():
    folium.CircleMarker(
        location=[row['Latitudine'], row['Longitudine']],
        radius=6,
        popup=f"<b>EEA:</b> {row['Stazione']}",
        color='red',
        fill=True,
        fillColor='red',
        fillOpacity=0.6,
        weight=1
    ).add_to(m)

# Draw lines connecting Trentino stations to their assigned EEA stations
for idx, row in proximity_mapping.iterrows():
    trentino_name = row['Trentino_Stazione']
    eea_name = row['EEA_Stazione']
    
    # Get coordinates
    trentino_coords = trentino_stations[trentino_stations['Stazione'] == trentino_name]
    eea_coords = eea_stations[eea_stations['Stazione'] == eea_name]
    
    if len(trentino_coords) > 0 and len(eea_coords) > 0:
        t_lat, t_lon = trentino_coords.iloc[0][['Latitudine', 'Longitudine']]
        e_lat, e_lon = eea_coords.iloc[0][['Latitudine', 'Longitudine']]
        
        folium.PolyLine(
            locations=[[t_lat, t_lon], [e_lat, e_lon]],
            color='green',
            weight=1,
            opacity=0.4
        ).add_to(m)

# Add legend
legend_html = '''
     <div style="position: fixed; 
                     bottom: 50px; left: 50px; width: 220px; height: 140px; 
                     background-color: white; border:2px solid grey; z-index:9999; 
                     font-size:14px; padding: 10px">
     <p style="margin:0; font-weight: bold;">Legend</p>
     <p style="margin:5px 0;"><i style="background:blue; width:15px; height:15px; 
        display:inline-block; border-radius:50%;"></i> Trentino Weather Stations</p>
     <p style="margin:5px 0;"><i style="background:red; width:12px; height:12px; 
        display:inline-block; border-radius:50%;"></i> EEA Air Quality Stations</p>
     <p style="margin:5px 0;"><i style="background:green; width:20px; height:2px; 
        display:inline-block;"></i> Station Proximity Links</p>
     </div>
     '''
m.get_root().html.add_child(folium.Element(legend_html))

m

## 8. Verify EEA Coordinates from Filtered CSV

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
eea_filtered_csv = pd.read_csv('../output/eea_filtered_by_proximity.csv')

In [ ]:
# Extract unique station coordinates from the filtered CSV
unique_stations_csv = eea_filtered_csv[['Stazione', 'Latitudine', 'Longitudine']].drop_duplicates().reset_index(drop=True)

print(f"Total unique EEA stations in filtered CSV: {len(unique_stations_csv)}")
print("\nUnique Stations:")
print(unique_stations_csv.to_string(index=False))

# Create interactive map with unique stations
center_lat = unique_stations_csv['Latitudine'].mean()
center_lon = unique_stations_csv['Longitudine'].mean()

m_filtered = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=9,
    tiles='OpenStreetMap'
)

# Add each unique station as a marker
for idx, row in unique_stations_csv.iterrows():
    folium.CircleMarker(
        location=[row['Latitudine'], row['Longitudine']],
        radius=8,
        popup=f"<b>Station:</b> {row['Stazione']}<br><b>Lat:</b> {row['Latitudine']:.5f}<br><b>Lon:</b> {row['Longitudine']:.5f}",
        color='purple',
        fill=True,
        fillColor='purple',
        fillOpacity=0.7,
        weight=2
    ).add_to(m_filtered)

# Add legend
legend_html = '''
     <div style="position: fixed; 
                     bottom: 50px; left: 50px; width: 220px; height: 100px; 
                     background-color: white; border:2px solid grey; z-index:9999; 
                     font-size:14px; padding: 10px">
     <p style="margin:0; font-weight: bold;">EEA Filtered Stations</p>
     <p style="margin:5px 0;"><i style="background:purple; width:15px; height:15px; 
        display:inline-block; border-radius:50%;"></i> Unique EEA Stations</p>
     <p style="margin:5px 0; font-size:12px;">Total: {}</p>
     </div>
     '''.format(len(unique_stations_csv))
m_filtered.get_root().html.add_child(folium.Element(legend_html))

m_filtered